# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [54]:
import numpy as np
import pandas as pd
import xlsxwriter
import math
from scipy import stats
import yfinance as yf

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks_new.csv')

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [83]:
stock = 'ACN'
data = yf.Ticker(stock)

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [151]:
stock = 'AXP'
data = yf.Ticker(stock)
data.financials

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Tax Effect Of Unusual Items,-36337000.0,-30672000.0,2460000.0,-33750000.0
Tax Rate For Calcs,0.203,0.216,0.246,0.27
Total Unusual Items,-179000000.0,-142000000.0,10000000.0,-125000000.0
Total Unusual Items Excluding Goodwill,-179000000.0,-142000000.0,10000000.0,-125000000.0
Net Income From Continuing Operation Net Minority Interest,8374000000.0,7514000000.0,8060000000.0,3135000000.0
Reconciled Depreciation,1651000000.0,1626000000.0,1695000000.0,1543000000.0
Net Interest Income,13134000000.0,9895000000.0,7750000000.0,7985000000.0
Interest Expense,6849000000.0,2763000000.0,1283000000.0,2098000000.0
Interest Income,19983000000.0,12658000000.0,9033000000.0,10083000000.0
Normalized Income,8516663000.0,7625328000.0,8052460000.0,3226250000.0


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [3]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [4]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = yf.Ticker(symbol_string)
    for symbol in symbol_string.split(','):
        try:
            data = yf.Ticker(symbol)
            price = data.info['currentPrice']
            pe_ratio = data.info['forwardPE']
            new_data = pd.Series([symbol, price, pe_ratio,'N/A'], index= my_columns)
            final_dataframe = pd.concat([final_dataframe, new_data.to_frame().T], ignore_index=True)
        except:
            print(symbol + " can't find")

BRK.B can't find
BF.B can't find
NWS can't find


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [5]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe_top50 = final_dataframe[:50]
final_dataframe_top50.reset_index(inplace= True)
final_dataframe_top50.drop('index', axis = 1 , inplace= True)
final_dataframe_top50

/var/folders/_h/v0jk_rtd6zzcy9tw3llczrf00000gn/T/ipykernel_49585/2299492364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe_top50.drop('index', axis = 1 , inplace= True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,VTRS,11.15,4.084249,N/A
1,AAL,14.475,4.595238,N/A
2,UAL,54.95,4.641047,N/A
3,GM,45.2401,4.663928,N/A
4,EG,389.83,5.642351,N/A
5,WBA,17.94,5.731629,N/A
6,APA,30.875,6.187375,N/A
7,BMY,43.655,6.308526,N/A
8,F,12.1898,6.382094,N/A
9,GL,85.42,6.642302,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [6]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [7]:
portfolio_input()

Enter the value of your portfolio: 1_000_000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size) / len(final_dataframe_top50.index)
for row in final_dataframe_top50.index:
    final_dataframe_top50.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe_top50.loc[row, 'Price'])
final_dataframe_top50

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,VTRS,11.15,4.084249,1793
1,AAL,14.475,4.595238,1381
2,UAL,54.95,4.641047,363
3,GM,45.2401,4.663928,442
4,EG,389.83,5.642351,51
5,WBA,17.94,5.731629,1114
6,APA,30.875,6.187375,647
7,BMY,43.655,6.308526,458
8,F,12.1898,6.382094,1640
9,GL,85.42,6.642302,234


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [9]:
# data.info #'priceToSalesTrailing12Months', 'forwardPE', 'priceToBook', 'enterpriseToEbitda', 
# data.financials # data.info['enterpriseValue'] /['Gross Profit']['2023-09-30']

In [14]:
rv_column = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales-Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_dataframe = pd.DataFrame(columns = rv_column)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [15]:
def ebitda(data):
    operating_income = data.info.get('operatingIncome', None)
    depreciation_amortization = data.info.get('depreciationAndAmortization', None)
    
    # Check if operating income and depreciation & amortization are available
    if operating_income is not None and depreciation_amortization is not None:
        # Calculate EBITDA
        ebitda = operating_income + depreciation_amortization
        print(f"EBITDA for {ticker_symbol}: {ebitda}")
    else:
        print("Operating income or Depreciation & Amortization data not available.")

In [42]:
rv_dataframe = pd.DataFrame(columns = rv_column)

for symbol_string in symbol_strings:
    for symbol in symbol_string.split(','):
        try:
            data1 = yf.Ticker(symbol)
            price = data1.info['currentPrice']
            num_shares = 'N/A'
            try:
                pe_ratio = data1.info['forwardPE']
            except:
                pe_ratio = np.NaN
            pe_percentile = 'N/A'
            price_to_sales = price / data1.info['revenuePerShare']
            ps_percentile = 'N/A'
            try:
                price_to_book = price / data1.info['bookValue']
            except:
                price_to_book = np.NaN
            pb_percentile = 'N/A' 
            financials1 = data1.financials
            try:
                gp = financials1.loc['Gross Profit', financials1.columns[0]]
            except:
                gp = np.NaN
            try:
                ebitda = financials1.loc['EBITDA', financials1.columns[0]]
            except:
                ebitda = np.NaN
            try:
                ev_ebitda = data1.info['enterpriseValue'] / ebitda
            except:
                ev_ebitda = np.NaN
            ev_ebitda_percentile = 'N/A'
            try:  
                ev_gp = data1.info['enterpriseValue'] / gp
            except:
                ev_gp = np.NaN
            ev_gp_percentile = 'N/A'
            rv_score = 'N/A'
            new_data = pd.Series([symbol, price,num_shares, pe_ratio, pe_percentile, price_to_book, pb_percentile,
                                   price_to_sales, ps_percentile, ev_ebitda, ev_ebitda_percentile,
                                   ev_gp, ev_gp_percentile, rv_score], index=rv_column)
            rv_dataframe = pd.concat([rv_dataframe, new_data.to_frame().T], ignore_index=True)
        except Exception as e:
            print(symbol + " can't be found:", e)

BRK.B can't be found: 'currentPrice'
BF.B can't be found: 'currentPrice'


In [43]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,104.91,N/A,13.501931,N/A,11.929725,N/A,1.781457,N/A,-10.297205,N/A,4.899944,N/A,N/A
1,AOS,86.07,N/A,19.786207,N/A,6.708496,N/A,3.318425,N/A,15.063966,N/A,8.357823,N/A,N/A
2,ABT,103.53,N/A,20.142023,N/A,4.640728,N/A,4.466158,N/A,17.959555,N/A,8.555415,N/A,N/A
3,ABBV,165.1704,N/A,13.707087,N/A,36.421257,N/A,5.367032,N/A,20.393996,N/A,10.329638,N/A,N/A
4,ACN,306.01,N/A,23.341724,N/A,7.094732,N/A,2.983688,N/A,17.921163,N/A,9.152321,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,ZBRA,319.83,N/A,21.566418,N/A,5.187498,N/A,3.773449,N/A,28.763163,N/A,8.711594,N/A,N/A
499,ZBH,119.33,N/A,13.700345,N/A,1.948404,N/A,3.324974,N/A,13.670048,N/A,5.715246,N/A,N/A
500,ZTS,173.74,N/A,27.18936,N/A,15.696088,N/A,9.146618,N/A,22.979195,N/A,14.080182,N/A,N/A
501,MMM,104.945,N/A,13.506435,N/A,11.933705,N/A,1.782051,N/A,-10.297205,N/A,4.899944,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [44]:
rv_dataframe[rv_dataframe.isnull().any(axis= 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
8,AFL,87.72,N/A,12.713043,N/A,2.12536,N/A,2.663266,N/A,NaN,N/A,NaN,N/A,N/A
18,ALL,167.815,N/A,9.573017,N/A,2.661533,N/A,0.752156,N/A,NaN,N/A,NaN,N/A,N/A
27,AXP,242.72,N/A,16.268097,N/A,6.06709,N/A,3.111954,N/A,NaN,N/A,NaN,N/A,N/A
28,AIG,79.0228,N/A,10.015564,N/A,1.222223,N/A,1.152642,N/A,NaN,N/A,NaN,N/A,N/A
31,AMP,432.2198,N/A,11.320581,N/A,8.827298,N/A,2.722885,N/A,NaN,N/A,NaN,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,USB,41.145,N/A,9.590909,N/A,1.316724,N/A,2.534808,N/A,NaN,N/A,NaN,N/A,N/A
466,VLTO,99.09,N/A,27.448753,N/A,NaN,N/A,4.845477,N/A,21.683505,N/A,9.06656,N/A,N/A
476,WRB,78.55,N/A,12.159443,N/A,2.58865,N/A,1.691685,N/A,NaN,N/A,NaN,N/A,N/A
486,WFC,60.85,N/A,11.003616,N/A,1.312582,N/A,2.847983,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [45]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales-Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column] = rv_dataframe[column].fillna(rv_dataframe[column].mean())

/var/folders/_h/v0jk_rtd6zzcy9tw3llczrf00000gn/T/ipykernel_49585/1803005620.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rv_dataframe[column] = rv_dataframe[column].fillna(rv_dataframe[column].mean())


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [47]:
rv_dataframe[rv_dataframe.isnull().any(axis= 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [127]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales-Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric, percentile_column in metrics.items():
    # Calculate percentile rank for each metric
    percentile_rank = stats.percentileofscore(rv_dataframe[metric], rv_dataframe[metric])
    # Assign the percentile rank to the corresponding column
    rv_dataframe[percentile_column] = percentile_rank / 100
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
479,WBA,17.915,2790,5.723642,0.015905,1.146193,0.111332,0.106928,0.001988,-19.951638,0.007952,1.902893,0.009940,2.942346
482,WBD,8.09,6180,-115.571434,0.001988,0.448746,0.071571,0.486207,0.047714,2.699693,0.019881,3.595357,0.053678,3.89662
25,AAL,14.485,3451,4.598413,0.007952,-1.727696,0.067594,0.178327,0.007952,7.360183,0.091451,3.440153,0.045726,4.413519
136,CVS,57.3903,871,7.357731,0.027833,0.971400,0.091451,0.204126,0.009940,7.764737,0.095427,2.595854,0.021869,4.930417
238,HPQ,31.805,1572,8.713698,0.057654,-19.022129,0.033797,0.595020,0.059642,9.020999,0.137177,3.398161,0.043738,6.640159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,PANW,320.2,N/A,51.728596,0.980119,23.715005,0.956262,13.193243,0.964215,116.813386,0.996024,20.371429,0.924453,96.421471
145,DXCM,130.42,N/A,57.707966,0.986083,22.993653,0.952286,13.271599,0.966203,56.651956,0.968191,22.688998,0.954274,96.540755
412,NOW,771.38,N/A,47.469540,0.970179,19.541965,0.938370,16.648968,0.986083,96.611196,0.992048,21.843723,0.946322,96.66004
324,MPWR,750.65,N/A,44.868504,0.960239,17.284533,0.932406,19.695380,0.996024,65.634981,0.982107,33.546687,0.990060,97.2167


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [128]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_dataframe.loc[row,metrics[metric]])
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentile)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
479,WBA,17.915,2790,5.723642,0.015905,1.146193,0.111332,0.106928,0.001988,-19.951638,0.007952,1.902893,0.009940,0.029423
482,WBD,8.09,6180,-115.571434,0.001988,0.448746,0.071571,0.486207,0.047714,2.699693,0.019881,3.595357,0.053678,0.038966
25,AAL,14.485,3451,4.598413,0.007952,-1.727696,0.067594,0.178327,0.007952,7.360183,0.091451,3.440153,0.045726,0.044135
136,CVS,57.3903,871,7.357731,0.027833,0.971400,0.091451,0.204126,0.009940,7.764737,0.095427,2.595854,0.021869,0.049304
238,HPQ,31.805,1572,8.713698,0.057654,-19.022129,0.033797,0.595020,0.059642,9.020999,0.137177,3.398161,0.043738,0.066402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,PANW,320.2,N/A,51.728596,0.980119,23.715005,0.956262,13.193243,0.964215,116.813386,0.996024,20.371429,0.924453,0.964215
145,DXCM,130.42,N/A,57.707966,0.986083,22.993653,0.952286,13.271599,0.966203,56.651956,0.968191,22.688998,0.954274,0.965408
412,NOW,771.38,N/A,47.469540,0.970179,19.541965,0.938370,16.648968,0.986083,96.611196,0.992048,21.843723,0.946322,0.9666
324,MPWR,750.65,N/A,44.868504,0.960239,17.284533,0.932406,19.695380,0.996024,65.634981,0.982107,33.546687,0.990060,0.972167


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [129]:
rv_dataframe.sort_values('RV Score', inplace=True)
rv_dataframe_top50 = rv_dataframe[:50]
rv_dataframe_top50.reset_index(drop=True, inplace=True)
rv_dataframe_top50

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,WBA,17.915,2790,5.723642,0.015905,1.146193,0.111332,0.106928,0.001988,-19.951638,0.007952,1.902893,0.009940,0.029423
1,WBD,8.09,6180,-115.571434,0.001988,0.448746,0.071571,0.486207,0.047714,2.699693,0.019881,3.595357,0.053678,0.038966
2,AAL,14.485,3451,4.598413,0.007952,-1.727696,0.067594,0.178327,0.007952,7.360183,0.091451,3.440153,0.045726,0.044135
3,CVS,57.3903,871,7.357731,0.027833,0.971400,0.091451,0.204126,0.009940,7.764737,0.095427,2.595854,0.021869,0.049304
4,HPQ,31.805,1572,8.713698,0.057654,-19.022129,0.033797,0.595020,0.059642,9.020999,0.137177,3.398161,0.043738,0.066402
5,HPE,17.776,2812,8.713725,0.059642,1.078837,0.107356,0.822773,0.103380,6.634346,0.071571,3.241039,0.041750,0.07674
6,UAL,54.93,910,4.639358,0.009940,1.965717,0.298211,0.328641,0.023857,4.970164,0.033797,2.561111,0.019881,0.077137
7,VTRS,11.105,4502,4.067765,0.005964,0.660501,0.075547,0.866427,0.115308,8.581948,0.123260,4.687109,0.103380,0.084692
8,BG,102.23,489,10.410388,0.147117,1.371810,0.163022,0.261052,0.017893,4.580398,0.027833,3.798563,0.069583,0.085089
9,T,17.505,2856,7.644105,0.033797,1.200617,0.123260,1.028556,0.133201,6.382358,0.059642,4.001188,0.081511,0.086282


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [130]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [131]:
position_size = float(portfolio_size) / len(rv_dataframe_top50)
for row in rv_dataframe_top50.index:
    rv_dataframe_top50.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe_top50.loc[row, 'Price'])
rv_dataframe_top50

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales-Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,WBA,17.915,2790,5.723642,0.015905,1.146193,0.111332,0.106928,0.001988,-19.951638,0.007952,1.902893,0.009940,0.029423
1,WBD,8.09,6180,-115.571434,0.001988,0.448746,0.071571,0.486207,0.047714,2.699693,0.019881,3.595357,0.053678,0.038966
2,AAL,14.485,3451,4.598413,0.007952,-1.727696,0.067594,0.178327,0.007952,7.360183,0.091451,3.440153,0.045726,0.044135
3,CVS,57.3903,871,7.357731,0.027833,0.971400,0.091451,0.204126,0.009940,7.764737,0.095427,2.595854,0.021869,0.049304
4,HPQ,31.805,1572,8.713698,0.057654,-19.022129,0.033797,0.595020,0.059642,9.020999,0.137177,3.398161,0.043738,0.066402
5,HPE,17.776,2812,8.713725,0.059642,1.078837,0.107356,0.822773,0.103380,6.634346,0.071571,3.241039,0.041750,0.07674
6,UAL,54.93,910,4.639358,0.009940,1.965717,0.298211,0.328641,0.023857,4.970164,0.033797,2.561111,0.019881,0.077137
7,VTRS,11.105,4502,4.067765,0.005964,0.660501,0.075547,0.866427,0.115308,8.581948,0.123260,4.687109,0.103380,0.084692
8,BG,102.23,489,10.410388,0.147117,1.371810,0.163022,0.261052,0.017893,4.580398,0.027833,3.798563,0.069583,0.085089
9,T,17.505,2856,7.644105,0.033797,1.200617,0.123260,1.028556,0.133201,6.382358,0.059642,4.001188,0.081511,0.086282


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [132]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe_top50.to_excel(writer, sheet_name = 'Value Strategy', index= False)

In [133]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_format = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )


percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [134]:
column_format = {
    'A':['Ticker',string_format],
    'B':['Price',dollar_format],
    'C':['Number of Shares to Buy',integer_format],
    'D':['Price-to-Earnings Ratio',float_format],
    'E':['PE Percentile',percent_format],
    'F':['Price-to-Book Ratio',float_format],
    'G':['PB Percentile',percent_format],
    'H':['Price-to-Sales-Ratio',float_format],
    'I':['PS Percentile',percent_format],
    'J':['EV/EBITDA',float_format],
    'K':['EV/EBITDA Percentile',percent_format],
    'L':['EV/GP',float_format],
    'M':['EV/GP Percentile',percent_format],
    'N':['RV Score',float_format]
    }

In [135]:
for column in column_format.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_format[column][1])

In [136]:
writer.close()